# Basic Chatbot

In [20]:
# Store Access Token as an environment variable
import getpass
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [33]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 24, 'total_tokens': 28, 'completion_time': 0.003333333, 'prompt_time': 0.004506276, 'queue_time': 0.166631092, 'total_time': 0.007839609}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--4e0f0d25-4d22-4e98-84f6-2e2addb2411e-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28})

# Search Engine

## Loading up search engine

We will be using Tavily (a search engine) as a tool. In order to use it, you will need to get and set an API key:
https://tavily.com/

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [26]:
# Loading up search engine 
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in hk")
print(search_results)

C:\Users\chinh\AppData\Local\Temp\ipykernel_3796\3510116346.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=2)


[{'title': 'Hong Kong Weather Forecast 2025-10-02', 'url': 'https://www.navigator-insurance.com/blog/hong-kong-weather-forecast-2025-10-02/', 'content': 'Sea Temperature (North Point): 28°C (Recorded: Oct 02, 2025, 07:00)\n\nSoil Temperature (Hong Kong Observatory, 0.5m): 29.7°C (Recorded: Oct 02, 2025, 07:00)\n\nSoil Temperature (Hong Kong Observatory, 1m): 29.2°C (Recorded: Oct 02, 2025, 07:00) [...] Mainly cloudy with a few squally showers and thunderstorms. More showers at first. Seas will be rough with swells.\n\nMax Temp: 29°C\n\nMin Temp: 26°C\n\nHumidity: 80-95%\n\nWind: East to southeast force 5 to 6, occasionally force 8 offshore and on high ground at first.\n\nProbability of Significant Rain: High\n\n### Monday, Oct 06\n\nMainly cloudy with a few showers. Sunny periods during the day.\n\nMax Temp: 31°C\n\nMin Temp: 27°C\n\nHumidity: 70-95%\n\nWind: East to southeast force 3 to 4. [...] ### Friday, Oct 03\n\nMainly fine. Very hot during the day.\n\nMax Temp: 33°C\n\nMin Temp:

## Combine search engine with LLM by agent

In [27]:
tools = [search]
model_with_tools = model.bind_tools(tools)

In [28]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF? You should use web search.")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': "What's the weather in San Francisco?"}, 'id': 'call_y72h', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

In [29]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [32]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the weather in SF?")]}
)
response["messages"]

[HumanMessage(content="What's the weather in SF?", additional_kwargs={}, response_metadata={}, id='a297c731-228b-4861-b79c-349f4a9b64f3'),
 AIMessage(content='The current weather in San Francisco is sunny with a high of 68°F (20°C) and a low of 56°F (13°C).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 1912, 'total_tokens': 1943, 'completion_time': 0.025833333, 'prompt_time': 0.237873462, 'queue_time': -0.294430081, 'total_time': 0.263706795}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--9c543b6e-402f-4537-99c5-85fd84227f4f-0', usage_metadata={'input_tokens': 1912, 'output_tokens': 31, 'total_tokens': 1943})]

The above response indicate the model do not use tavily. So, we should add SystemMessage to instruct the agent to use web search if needed

In [39]:
# Adding SystemMessage to instruct the agent to use web search if needed
response = agent_executor.invoke(
    {"messages": [SystemMessage(content='Whenever you do not know the answer or need up-to-date information, perform a web search to find accurate and relevant results before responding.'), 
                  HumanMessage(content="What's the weather in SF? ")]}
)
response["messages"]

[SystemMessage(content='Whenever you do not know the answer or need up-to-date information, perform a web search to find accurate and relevant results before responding.', additional_kwargs={}, response_metadata={}, id='6ea97751-6268-4b30-9866-b5328952a886'),
 HumanMessage(content="What's the weather in SF? ", additional_kwargs={}, response_metadata={}, id='0b671691-727e-4040-86d3-d2bf8f52e683'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xpz7', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 1975, 'total_tokens': 2055, 'completion_time': 0.066666667, 'prompt_time': 0.246168528, 'queue_time': -0.302638948, 'total_time': 0.312835195}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9eda1e98-4db8-4a97-98df-f1ff51c1274f-0', too

## Adding in memory

The agent is stateless without MemorySaver. To make it knows which thread/conversation to resume from, we also have to pass in a thread_id when invoking the agent. 
Use LangGraph SqliteSaver or PostgresSaver and connect a database for persistent store. 

In [40]:
from langgraph.checkpoint.memory import MemorySaver

# In-Memory Saver, for demo only
memory = MemorySaver()

In [41]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Change thread_id for different conversation
config = {"configurable": {"thread_id": "abc123"}}

In [42]:
# 1st conversation
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 1906, 'total_tokens': 1910, 'completion_time': 0.003333333, 'prompt_time': 0.237213137, 'queue_time': -0.289303203, 'total_time': 0.24054647}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run--86f814cc-e641-44a9-81ef-828fc8c82ab5-0', usage_metadata={'input_tokens': 1906, 'output_tokens': 4, 'total_tokens': 1910})]}}
----


In [43]:
# 2nd conversation
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 961, 'total_tokens': 967, 'completion_time': 0.005, 'prompt_time': 0.119936318, 'queue_time': 0.166901968, 'total_time': 0.124936318}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--6cea7bf1-1f1f-42f5-88b7-c3f43cec57b5-0', usage_metadata={'input_tokens': 961, 'output_tokens': 6, 'total_tokens': 967})]}}
----


The agent replied "Your name is Bob!" which means it remember the conversation. 

### PostgreSQL as Memory Store

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the database URL
DATABASE_URL = os.getenv('DATABASE_URL')

In [28]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.postgres import PostgresSaver
memory = PostgresSaver.from_conn_string(DATABASE_URL)
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [29]:
# Change thread_id for different conversation
config = {"configurable": {"thread_id": "31025a"}}

In [34]:
from langchain_core.runnables import RunnableConfig
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.postgres import PostgresSaver
#from langgraph.store.postgres import PostgresStore
#from langgraph.store.base import BaseStore
import uuid

with (
    #PostgresStore.from_conn_string(DATABASE_URL) as store,
    PostgresSaver.from_conn_string(DATABASE_URL) as checkpointer,
):
    #store.setup()
    #checkpointer.setup()

    def call_model(
        state: MessagesState,
        config: RunnableConfig,
        #store: BaseStore,
    ):
        return {"messages": [model.invoke(state["messages"])]}

    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_edge(START, "call_model")

    graph = builder.compile(
        checkpointer=checkpointer,
        #store=store,
    )

    config = {
        "configurable": {
            "thread_id": "31025a",
            "user_id": "1",
        }
    }
    
    for chunk in graph.stream(
        {"messages": [{"role": "user", "content": "Hi! Remember: my name is Bob"}]},
        config,
        stream_mode="values",
    ):
        chunk["messages"][-1].pretty_print()

    for chunk in graph.stream(
        {"messages": [{"role": "user", "content": "what is my name?"}]},
        config,
        stream_mode="values",
    ):
        chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! Remember: my name is Bob
================================== Ai Message ==================================

Hello Bob, nice to see you again. How can I assist you today?
================================ Human Message =================================

what is my name?
================================== Ai Message ==================================

Your name is Bob.


In [38]:
from langgraph.checkpoint.postgres import PostgresSaver
with PostgresSaver.from_conn_string(DATABASE_URL) as memory:
    # Run a graph, then list the checkpoints
    #config = {"configurable": {"thread_id": "1"}}
    checkpoints = list(memory.list(config))

In [37]:
checkpoints[0].checkpoint

{'v': 4,
 'id': '1f0a0103-5739-623f-8006-329f44118594',
 'ts': '2025-10-03T04:19:52.517279+00:00',
 'versions_seen': {'__input__': {},
  '__start__': {'__start__': '00000000000000000000000000000006.0.845301009096755'},
  'call_model': {'branch:to:call_model': '00000000000000000000000000000007.0.6912299745660969'}},
 'channel_values': {'messages': [HumanMessage(content='Hi! Remember: my name is Bob', additional_kwargs={}, response_metadata={}, id='bc6c2b8c-4530-488b-bfe3-55fa8bf22d7e'),
   HumanMessage(content='Hi! Remember: my name is Bob', additional_kwargs={}, response_metadata={}, id='2c9d415b-429f-4c58-9517-ca41ef04f1fc'),
   AIMessage(content='Hello Bob, nice to see you again. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 56, 'total_tokens': 73, 'completion_time': 0.052048593, 'prompt_time': 0.003370532, 'queue_time': 4.246878481, 'total_time': 0.055419125}, 'model_name': 'llama-3.3-70b-versatile', 